In [2]:
%cd practicas/pr402/

/media/notebooks/practicas/pr402


/usr/local/lib/python3.10/dist-packages/IPython/core/magics/osm.py:417: UserWarning: This is now an optional IPython functionality, setting dhist requires you to install the `pickleshare` library.
  self.shell.db['dhist'] = compress_dhist(dhist)[-100:]


# Ejercicio 1

In [2]:
%%writefile mapper1.py
#!/usr/bin/env python3

import sys

first = True

for line in sys.stdin:
    if first:
        first = False
        continue  # saltar cabecera
    line = line.strip().split(",")
    print(f"{line[3]}\t{line[7]}")



Overwriting mapper1.py


In [3]:
%%writefile reducer1.py
#!/usr/bin/env python3

import sys

current_city = None
current_temperature = 0

for line in sys.stdin:
    city, temperature = line.strip().split("\t", 1)
    temperature = float(temperature)
    
    if current_city is None:
        current_city = city
        current_temperature = temperature
    elif city == current_city:
        if temperature > current_temperature:
            current_temperature = temperature
    else:
        print(f"{current_city} ---> {current_temperature}")
        current_city = city
        current_temperature = temperature

if current_city is not None:
    print(f"{current_city} ---> {current_temperature}")


Overwriting reducer1.py


In [4]:
!hdfs dfs -mkdir /pr402

In [5]:
!hdfs dfs -put city_temperature.csv /pr402/

In [1]:
!find / -name hadoop-streaming*

/usr/local/hadoop/share/hadoop/tools/lib/hadoop-streaming-3.4.0.jar
/usr/local/hadoop/share/hadoop/tools/sources/hadoop-streaming-3.4.0-sources.jar
/usr/local/hadoop/share/hadoop/tools/sources/hadoop-streaming-3.4.0-test-sources.jar
/usr/local/hadoop/share/doc/hadoop/hadoop-streaming
/usr/local/hadoop/libexec/shellprofile.d/hadoop-streaming.sh
/usr/local/hadoop/libexec/tools/hadoop-streaming.sh


In [6]:
!hadoop jar \
/usr/local/hadoop/share/hadoop/tools/lib/hadoop-streaming-3.4.0.jar \
-file mapper1.py \
-file reducer1.py \
-mapper mapper1.py \
-reducer reducer1.py \
-input /pr402/city_temperature.csv \
-output /pr402/salida1


2025-11-21 13:30:35,735 WARN streaming.StreamJob: -file option is deprecated, please use generic option -files instead.
packageJobJar: [mapper1.py, reducer1.py, /tmp/hadoop-unjar7066145916777727139/] [] /tmp/streamjob3618978582188152663.jar tmpDir=null
2025-11-21 13:30:36,339 INFO client.DefaultNoHARMFailoverProxyProvider: Connecting to ResourceManager at yarnmanager/172.20.0.6:8032
2025-11-21 13:30:36,470 INFO client.DefaultNoHARMFailoverProxyProvider: Connecting to ResourceManager at yarnmanager/172.20.0.6:8032
2025-11-21 13:30:36,807 INFO mapreduce.JobResourceUploader: Disabling Erasure Coding for path: /tmp/hadoop-yarn/staging/root/.staging/job_1763731605970_0001
2025-11-21 13:30:37,169 INFO mapred.FileInputFormat: Total input files to process : 1
2025-11-21 13:30:37,200 INFO net.NetworkTopology: Adding a new node: /default-rack/172.20.0.2:9866
2025-11-21 13:30:37,200 INFO net.NetworkTopology: Adding a new node: /default-rack/172.20.0.3:9866
2025-11-21 13:30:37,201 INFO net.Network

In [ ]:
#!hdfs dfs -rm -r /pr402/salida1

Deleted /pr402/salida1


In [7]:
!hdfs dfs -cat /pr402/salida1/part-00000

Abidjan ---> 88.6	
Abilene ---> 94.2	
Abu Dhabi ---> 107.3	
Addis Ababa ---> 77.0	
Akron Canton ---> 86.1	
Albany ---> 88.0	
Albuquerque ---> 89.4	
Algiers ---> 96.6	
Allentown ---> 91.1	
Almaty ---> 90.9	
Amarillo ---> 92.5	
Amman ---> 95.4	
Amsterdam ---> 85.5	
Anchorage ---> 75.3	
Ankara ---> 87.9	
Antananarivo ---> 78.5	
Ashabad ---> 102.2	
Asheville ---> 85.1	
Athens ---> 94.3	
Atlanta ---> 92.8	
Atlantic City ---> 93.3	
Auckland ---> 75.4	
Austin ---> 94.5	
Baltimore ---> 91.9	
Bangkok ---> 93.0	
Bangui ---> 93.7	
Banjul ---> 93.6	
Barcelona ---> 86.6	
Baton Rouge ---> 90.2	
Beijing ---> 92.9	
Beirut ---> 91.1	
Belfast ---> 77.6	
Belgrade ---> 91.9	
Belize City ---> 92.9	
Bern ---> 83.7	
Bilbao ---> 94.6	
Billings ---> 90.6	
Birmingham ---> 91.0	
Bishkek ---> 91.1	
Bismarck ---> 91.7	
Bissau ---> 100.1	
Bogota ---> 66.7	
Boise ---> 94.2	
Bombay (Mumbai) ---> 92.6	
Bonn ---> 86.9	
Bordeaux ---> 88.8	
Boston ---> 90.7	
Brasilia ---> 87.7	
Bratislava ---> 85.5	
Brazzaville ---> 88.7

# Ejercicio 2 

In [8]:
%%writefile mapper2.py
#!/usr/bin/env python3

import sys

first = True

for line in sys.stdin:
    if first:
        first = False
        continue
    line = line.strip().split(",")
    print(f"{line[1]}\t{line[7]}")

Overwriting mapper2.py


In [9]:
%%writefile reducer2.py
#!/usr/bin/env python3

import sys

c_pais = None
c_temp = 0
count = 0
total = 0
media = 0


for line in sys.stdin:
    pais, temp = line.strip().split("\t", 1)
    temp = float(temp)

    if c_pais == None:
        c_pais = pais
        c_temp = temp
        count += 1
    elif c_pais == pais:
        c_temp = temp
        count += 1
        total += c_temp
    else:
        media = total / count
        print(f"{c_pais} ---> {media:.2f}")
        c_pais = pais
        c_temp = temp
        media = 0 
        total = c_temp
        count = 1
        
if c_pais is not None:
    print(f"{pais} ---> {media:.2f}")

Overwriting reducer2.py


In [15]:
!hadoop jar \
/usr/local/hadoop/share/hadoop/tools/lib/hadoop-streaming-3.4.0.jar \
-file mapper2.py \
-file reducer2.py \
-mapper mapper2.py \
-reducer reducer2.py \
-input /pr402/city_temperature.csv \
-output /pr402/salida2

2025-11-22 12:25:44,822 WARN streaming.StreamJob: -file option is deprecated, please use generic option -files instead.
packageJobJar: [mapper2.py, reducer2.py, /tmp/hadoop-unjar4362523181086264440/] [] /tmp/streamjob6733452254224717846.jar tmpDir=null
2025-11-22 12:25:45,400 INFO client.DefaultNoHARMFailoverProxyProvider: Connecting to ResourceManager at yarnmanager/172.20.0.6:8032
2025-11-22 12:25:45,517 INFO client.DefaultNoHARMFailoverProxyProvider: Connecting to ResourceManager at yarnmanager/172.20.0.6:8032
2025-11-22 12:25:45,695 INFO mapreduce.JobResourceUploader: Disabling Erasure Coding for path: /tmp/hadoop-yarn/staging/root/.staging/job_1763813424795_0002
2025-11-22 12:25:46,035 INFO mapred.FileInputFormat: Total input files to process : 1
2025-11-22 12:25:46,052 INFO net.NetworkTopology: Adding a new node: /default-rack/172.20.0.3:9866
2025-11-22 12:25:46,053 INFO net.NetworkTopology: Adding a new node: /default-rack/172.20.0.5:9866
2025-11-22 12:25:46,053 INFO net.Network

In [16]:
!hdfs dfs -cat /pr402/salida2/part-00000

Albania ---> 33.17	
Algeria ---> 63.76	
Argentina ---> 62.30	
Australia ---> 61.63	
Austria ---> 51.05	
Bahamas ---> 76.57	
Bahrain ---> 80.64	
Bangladesh ---> 10.11	
Barbados ---> 77.00	
Belarus ---> 41.82	
Belgium ---> 51.06	
Belize ---> 73.48	
Benin ---> 76.15	
Bermuda ---> 66.97	
Bolivia ---> 44.87	
Brazil ---> 70.13	
Bulgaria ---> 45.20	
Burundi ---> -65.40	
Canada ---> 42.01	
Central African Republic ---> 67.02	
China ---> 59.98	
Colombia ---> 55.24	
Congo ---> 69.32	
Costa Rica ---> 70.38	
Croatia ---> 46.93	
Cuba ---> 72.64	
Cyprus ---> 23.78	
Czech Republic ---> 47.60	
Denmark ---> 46.96	
Dominican Republic ---> 65.22	
Egypt ---> 71.95	
Equador ---> 59.95	
Ethiopia ---> 25.46	
Finland ---> 42.24	
France ---> 54.70	
Gabon ---> 70.40	
Gambia ---> 59.67	
Georgia ---> 46.52	
Germany ---> 6.42	
Greece ---> 59.63	
Guatemala ---> 56.91	
Guinea ---> 49.39	
Guinea-Bissau ---> 2.39	
Guyana ---> -22.10	
Haiti ---> 16.69	
Honduras ---> 68.53	
Hong Kong ---> 75.06	
Hungary ---> 51.10	
Icel

In [14]:
!hdfs dfs -rm -r /pr402/salida2

Deleted /pr402/salida2


# Ejercicio 3

In [13]:
%%writefile mapper3.py
#!/usr/bin/env python3

import sys

first = True

for line in sys.stdin:
    if first:
        first = False
        continue
    line = line.strip().split(",")
    print(f"{line[3]}\t{line[6]}\t{line[7]}")

Overwriting mapper3.py


In [5]:
%%writefile reducer3.py
#!/usr/bin/env python3

import sys

c_ciudad = None
ciudad = None
c_año = 0
año = 0
c_temp = 0
temp = 0
count = 0

for line in sys.stdin:
    line = line.strip().split("\t", 2)
    ciudad = line[0]
    año = line[1]
    temp = float(line[2])

    if c_ciudad == None:
        c_ciudad = ciudad
        c_año = año
        if ((temp - 32) * 5 / 9) >= 30:
            count += 1
    elif ciudad == c_ciudad:
        if año == c_año:
            if ((temp - 32) * 5 / 9) >= 30:
                count += 1
        else:
            print(f"{c_ciudad} ({c_año}) ---> {count}")
            c_año = año
            if ((temp - 32) * 5 / 9) >= 30:
                count = 1
            else:
                count = 0

    else:
        print(f"{c_ciudad} ({c_año}) ---> {count}")
        c_ciudad = ciudad
        c_año = año
        if ((temp - 32) * 5 / 9) >= 30:
            count = 1
        else:
            count = 0

if c_ciudad is not None:
    print(f"{ciudad} ({c_año}) ---> {count}")

Overwriting reducer3.py


In [7]:
!hadoop jar \
/usr/local/hadoop/share/hadoop/tools/lib/hadoop-streaming-3.4.0.jar \
-file mapper3.py \
-file reducer3.py \
-mapper mapper3.py \
-reducer reducer3.py \
-input /pr402/city_temperature.csv \
-output /pr402/salida3

2025-11-22 12:23:12,578 WARN streaming.StreamJob: -file option is deprecated, please use generic option -files instead.
packageJobJar: [mapper3.py, reducer3.py, /tmp/hadoop-unjar3122232274086997394/] [] /tmp/streamjob8793873097124749676.jar tmpDir=null
2025-11-22 12:23:13,308 INFO client.DefaultNoHARMFailoverProxyProvider: Connecting to ResourceManager at yarnmanager/172.20.0.6:8032
2025-11-22 12:23:13,499 INFO client.DefaultNoHARMFailoverProxyProvider: Connecting to ResourceManager at yarnmanager/172.20.0.6:8032
2025-11-22 12:23:13,880 INFO mapreduce.JobResourceUploader: Disabling Erasure Coding for path: /tmp/hadoop-yarn/staging/root/.staging/job_1763813424795_0001
2025-11-22 12:23:15,350 INFO mapred.FileInputFormat: Total input files to process : 1
2025-11-22 12:23:15,376 INFO net.NetworkTopology: Adding a new node: /default-rack/172.20.0.3:9866
2025-11-22 12:23:15,376 INFO net.NetworkTopology: Adding a new node: /default-rack/172.20.0.4:9866
2025-11-22 12:23:15,376 INFO net.Network

In [6]:
!hdfs dfs -rm -r /pr402/salida3

Deleted /pr402/salida3


In [8]:
!hdfs dfs -cat /pr402/salida3/part-00000

Abidjan (1995) ---> 0	
Abidjan (2020) ---> 7	
Abidjan (2019) ---> 11	
Abidjan (2018) ---> 0	
Abidjan (2017) ---> 3	
Abidjan (2016) ---> 7	
Abidjan (2015) ---> 1	
Abidjan (2014) ---> 0	
Abidjan (2013) ---> 0	
Abidjan (2012) ---> 0	
Abidjan (2011) ---> 0	
Abidjan (2010) ---> 24	
Abidjan (2009) ---> 0	
Abidjan (2008) ---> 0	
Abidjan (2007) ---> 0	
Abidjan (2006) ---> 1	
Abidjan (2005) ---> 19	
Abidjan (2004) ---> 5	
Abidjan (2003) ---> 8	
Abidjan (2002) ---> 0	
Abidjan (2001) ---> 2	
Abidjan (2000) ---> 1	
Abidjan (1999) ---> 2	
Abidjan (1998) ---> 19	
Abidjan (1997) ---> 0	
Abidjan (1996) ---> 1	
Abidjan (1995) ---> 4	
Abilene (2017) ---> 0	
Abilene (2018) ---> 31	
Abilene (2019) ---> 33	
Abilene (2020) ---> 1	
Abilene (1995) ---> 9	
Abilene (1996) ---> 21	
Abilene (1997) ---> 6	
Abilene (1998) ---> 49	
Abilene (1999) ---> 27	
Abilene (2000) ---> 52	
Abilene (2001) ---> 40	
Abilene (2002) ---> 5	
Abilene (2003) ---> 15	
Abilene (2004) ---> 2	
Abilene (2005) ---> 4	
Abilene (2006) ---> 35

# Ejercicio 4

In [16]:
%%writefile mapper4.py
#!/usr/bin/env python3

import sys

first = True

for line in sys.stdin:
    if first:
        first = False
        continue
    line = line.strip().split(",")
    print(f"{line[0]}\t{line[7]}")

Overwriting mapper4.py


In [17]:
%%writefile reducer4.py
#!/usr/bin/env python3

import sys

region = None
c_region = None
temp = 0
max = 0
min = 10000

for line in sys.stdin:
    line = line.strip().split("\t", 1)

    if len(line) == 2:
        region = line[0]
        temp = float(line[1])
    else:
        continue
    
    if c_region == None:
        c_region = region
        max = temp
        min = temp

    elif c_region == region:
        if temp > max:
            max = temp
        elif temp < min:
            min = temp
            
    else:
        print(f"---> {c_region}: MAX ({max}) MIN ({min})")
        c_region = region
        max = temp
        min = temp

if c_region is not None:
    print(f"---> {c_region}: MAX ({max}) MIN ({min})")

Overwriting reducer4.py


In [19]:
!hadoop jar \
/usr/local/hadoop/share/hadoop/tools/lib/hadoop-streaming-3.4.0.jar \
-file mapper4.py \
-file reducer4.py \
-mapper mapper4.py \
-reducer reducer4.py \
-input /pr402/city_temperature.csv \
-output /pr402/salida4

2025-11-26 11:19:16,565 WARN streaming.StreamJob: -file option is deprecated, please use generic option -files instead.
packageJobJar: [mapper4.py, reducer4.py, /tmp/hadoop-unjar5898766092738003375/] [] /tmp/streamjob284135076605718543.jar tmpDir=null
2025-11-26 11:19:17,205 INFO client.DefaultNoHARMFailoverProxyProvider: Connecting to ResourceManager at yarnmanager/172.20.0.4:8032
2025-11-26 11:19:17,312 INFO client.DefaultNoHARMFailoverProxyProvider: Connecting to ResourceManager at yarnmanager/172.20.0.4:8032
2025-11-26 11:19:17,481 INFO mapreduce.JobResourceUploader: Disabling Erasure Coding for path: /tmp/hadoop-yarn/staging/root/.staging/job_1764154782104_0003
2025-11-26 11:19:17,777 INFO mapred.FileInputFormat: Total input files to process : 1
2025-11-26 11:19:17,790 INFO net.NetworkTopology: Adding a new node: /default-rack/172.20.0.3:9866
2025-11-26 11:19:17,791 INFO net.NetworkTopology: Adding a new node: /default-rack/172.20.0.6:9866
2025-11-26 11:19:17,791 INFO net.NetworkT

In [20]:
!hdfs dfs -cat /pr402/salida4/part-00000

---> Africa: MAX (102.8) MIN (-99.0)	
---> Asia: MAX (103.7) MIN (-99.0)	
---> Australia/South Pacific: MAX (96.8) MIN (-99.0)	
---> Europe: MAX (102.5) MIN (-99.0)	
---> Middle East: MAX (110.0) MIN (-99.0)	
---> North America: MAX (107.7) MIN (-99.0)	
---> South/Central America & Carribean: MAX (97.4) MIN (-99.0)	


In [18]:
!hdfs dfs -rm -r /pr402/salida4

Deleted /pr402/salida4


In [21]:
!cat city_temperature.csv | python3 mapper4.py | sort | python3 reducer4.py

---> Africa: MAX (102.8) MIN (-99.0)
---> Asia: MAX (103.7) MIN (-99.0)
---> Australia/South Pacific: MAX (96.8) MIN (-99.0)
---> Europe: MAX (102.5) MIN (-99.0)
---> Middle East: MAX (110.0) MIN (-99.0)
---> North America: MAX (107.7) MIN (-99.0)
---> South/Central America & Carribean: MAX (97.4) MIN (-99.0)
